<a href="https://colab.research.google.com/github/MarkusThill/techdays25/blob/feature-lab1/notebooks/lab1-my-first-onnx-model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 Lab 1: Erstelle dein erstes ONNX-Modell Schritt für Schritt
- This notebook should also run on a non-GPU device

## Preparation: Setup and install Dependencies

In [ ]:
# Remove the `%%capture`, if you have the impression that something is going wrong during the setup
%%capture
!pip install "techdays25[lab1] @ git+https://github.com/MarkusThill/techdays25.git@feature-lab1"

In [ ]:
# @title Some Colab-Specific Configuration {display-mode: "form"}
import sys

IN_COLAB = "google.colab" in sys.modules

if IN_COLAB:
    from google.colab import output

    output.enable_custom_widget_manager()

## Einleitung

Notes:
- Assume that we record pairs of  using a

## Datengenerierung

In [ ]:
# Import the data generation module and generate a small 2-dimensional dataset
from techdays25 import measurement_utils as mu

df = mu.generate_measurement_data()

print("df.shape:", df.shape)

# Show the first few lines of the DataFrame
df.head()

In [ ]:
# @title Visualize the Generated Data {display-mode: "form"}
mu.plot_measurement_data(*df.to_numpy().T)

In [ ]:
# TODO: Add Total Sum of Residuals

In [ ]:
# @title Manually training a linear model {display-mode: "form"}

# This code will be hidden when the notebook is loaded.

%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt

# set up plot
fig, ax = plt.subplots(figsize=(10, 6))
fig.canvas.toolbar_visible = False
fig.canvas.resizable = False
fig.canvas.toolbar_visible = False
fig.canvas.header_visible = False
fig.canvas.footer_visible = False
fig.canvas.capture_scroll = True
ax.grid(True, which="both")

uu, ii = df.to_numpy().T

ax.scatter(uu, ii)

# plt.minorticks_on()
ax.set_xlabel(r"$U \ [V]$")
ax.set_ylabel(r"$I \ [mA]$")


def linear_func(x, m, b):
    """Return a ..."""
    return m * x + b


(line,) = ax.plot(uu, linear_func(uu, 0, 0), color="black")

dy = linear_func(uu, 0, 0) - ii
residuals_line = ax.vlines(uu, ii, ii + dy, color="darkred")

font = {
    "size": 14,
}
serr = sum(abs(dy) ** 2)
my_textbox = ax.text(
    1, 40, r"Summe der quadrierten Fehler (SSE):" + str(round(serr, 2)), fontdict=font
)  #


@widgets.interact(m=(0, 4, 0.01), b=(-10, 10, 0.01))
def update(m=1.0, b=1.0):
    """Remove old lines from plot and plot new one."""
    line.set_ydata(linear_func(uu, m, b))

    # Update line
    pred = linear_func(uu, m, b)
    dy = pred - ii
    seg = residuals_line.get_segments()
    new_seg = []
    for s, newMin, newMax in zip(seg, pred, pred - dy):
        s[0, -1] = newMin
        s[1, -1] = newMax
        new_seg.append(s)
    residuals_line.set_segments(new_seg)

    # Update text box
    serr = sum(abs(dy) ** 2)
    my_textbox.set_text(r"Summe der quadrierten Fehler (SSE):" + str(round(serr, 2)))

    fig.canvas.draw()

## Training unseres ersten Machine-Learning-Modells
### Variante 1: Lineare Regression mit SciKit-Learn
TODO: Explain scikit-learn here:

- Das Scikit-Learn Modell versucht, die Steigung `m` und den Y-Achsenabschnitt (Ordinatenabschnitt) `b` so zu wählen, dass die Summe der quadratischen Fehler minimiert wird

Weitere Details und Beispiele können hier nachgeschlagen werden:
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [ ]:
import numpy as np
from sklearn.linear_model import LinearRegression

# write the recorded voltages and currents into the two numpy arrays `u` and `i`
uu, ii = df.to_numpy().T

# Note that the input has to be reshaped to a matrix of shape (N, 1)
# since the LinearRegression model is designed to handle multivariate
# regression problems
U = uu.reshape(-1, 1)

# Initialize and train regressor
scikit_regressor = LinearRegression().fit(U, ii)

In [ ]:
# Get the fitted slope `m` and the intercept `b`
scikit_regressor.coef_, scikit_regressor.intercept_

In [ ]:
# Now let us put the original data through the model and plot the obtained predictions:

# put the Nx1-shaped input through the regression model
ii_scikit = scikit_regressor.predict(U)

# Plot the predicted line
mu.plot_measurement_data(
    uu,
    ii,
    ii_scikit,
    xx_pred=U.flatten(),
    title="Original Data & Predictions by Scikit-Learn",
)

#### Optionale Aufgaben
1. Stimmen die durch Scikit-Learn ermittelten Parameter `regressor.coef_, regressor.intercept_` in etwa mit den Werten `m` und `b` überein, die ihr im interaktivem Widget weiter oben ermittelt habt?
2. Wie könnten wir überprüfen, ob die ermittelten Parameter `m`, `b` schlüssig sind?
3. Könnt ihr für 5 neue beliebige Inputs ($U [V]$) auch die zugehörigen Ströme ($I [mA]$) prognostizieren und plotten?

#### Export des scikit-Learn Modells nach ONNX
TODO: Beschreibung

In [ ]:
from pathlib import Path

import onnx
import skl2onnx
from skl2onnx.common.data_types import FloatTensorType

# Define the initial type for the model's input
initial_type = [("float_input", FloatTensorType([None, U.shape[1]]))]

# Convert the model
onnx_model = skl2onnx.convert_sklearn(scikit_regressor, initial_types=initial_type)

# Save the ONNX model to a file
Path("skl_regression.onnx").write_bytes(onnx_model.SerializeToString())

# Load the ONNX model
onnx_model = onnx.load("skl_regression.onnx")

# Check the model
onnx.checker.check_model(onnx_model)

# Print a human-readable representation of the model
print(onnx.helper.printable_graph(onnx_model.graph))

#### Visualisierung des ONNX-Graphen mit Netron
- ONNX-Modelle können auf [https://netron.app/](https://netron.app/) visualisiert werden
- Alternativ lässt sich auch das `netron` Python Package in Google Colab nutzen

### Inferenz mit dem scikit-learn ONNX Modell

In [ ]:
import onnxruntime as ort

# Load the ONNX model
# Specify the path to the ONNX model file
onnx_model_path = "linear_regression.onnx"

# Create an InferenceSession to load the model
session = ort.InferenceSession(onnx_model_path)

# Example input data (make sure it matches the input shape expected by the model)
# Let us create some new input data. Here, we create a NumPy array with shape
# (10, 1) representing 10 samples with 1 feature (voltage) each.
# We sample 10 equidistant points between 0 and 15.
U_new = np.linspace(start=-2, stop=15, num=10, dtype=np.float32).reshape(-1, 1)

# Get the name of the input node
# The model may have multiple inputs, but we usually need the first one
input_name = session.get_inputs()[0].name

# Get the name of the output node
# Similarly, the model may have multiple outputs, but we usually need the first one
output_name = session.get_outputs()[0].name

# Run inference
# The run method takes a list of output node names and a dictionary mapping input node names to input data
# It returns a list of outputs corresponding to the output node names provided
ii_sklearn_onnx = session.run([output_name], {input_name: U_new})

# Plot the predictions
# The predictions are returned as a list of NumPy arrays, so we print the first (and only) element
mu.plot_measurement_data(
    uu,
    ii,
    ii_sklearn_onnx[0].flatten(),
    xx_pred=U_new.flatten(),
    title="Original data & predictions by the ONNX model exported from scikit-learn",
)

#### Optionale Aufgaben:
1. Was passiert in obigem Beispiel wenn wir in der Zeile 
   `U_new = np.linspace(start=0, stop=15, num=10, dtype=np.float32).reshape(-1,1) `
   das Argument `dtype=np.float32` ersetzen durch `dtype=np.float64`?
2. 


### Selbstständiges "Training" des Modells durch uns

In [ ]:
import sympy

# Lets get our original dataset again:
uu, ii = df.to_numpy().T

U = np.stack([uu, np.ones(len(uu))]).T
sympy.Matrix(U[:5])